# Cycle Manager API - Complete Workflows Demo

This notebook demonstrates all the workflows in the Cycle Manager system:

1. **Admin Workflow**: Create and activate a review cycle
2. **Reviewer Workflow**: Complete evaluations
3. **Approver Workflow**: Review and approve evaluations
4. **Subject Workflow**: View reviews (read-only)

## Setup


In [7]:
import sys
from pathlib import Path

# Add the project root to the path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from cycle_manager_client import CycleManagerClient
from cycle_manager_client.models import AssignmentCreate

# Configuration
BASE_URL = "http://127.0.0.1:8000"

# API prefix - the client will auto-detect, but you can override:
# - "/api" if your server uses /api/cycles, /api/me, etc.
# - "" if your server uses /cycles, /me, etc. directly
API_PREFIX = "/api"  # Change to "" if your server doesn't use /api prefix

# User emails (adjust based on your test data)
ADMIN_EMAIL = "admin@local.test"
REVIEWER_EMAIL = "reviewer@local.test"
APPROVER_EMAIL = "approver@local.test"

print("✅ Imports successful!")


✅ Imports successful!


## 1. Admin Workflow: Create and Activate Cycle


In [8]:
# Initialize admin client
admin = CycleManagerClient(
    base_url=BASE_URL,
    user_email=ADMIN_EMAIL,
    api_prefix=API_PREFIX  # Use detected or configured prefix
)

print(f"🔧 Admin client initialized (API prefix: '{admin.api_prefix}')")


🔧 Admin client initialized (API prefix: '/api')


In [9]:
# Step 1: Create review cycle
print("\n[ADMIN] Step 1: Creating review cycle...")
# Note: Some server configurations require start_date and end_date
from datetime import date
cycle = admin.cycles.create(
    name="Q4 2025 Performance Reviews",
    start_date="2024-10-01",  # ISO format: YYYY-MM-DD
    end_date="2024-12-31"
)
print(f"   ✅ Created cycle: {cycle.id}")
print(f"   Name: {cycle.name}")
print(f"   Status: {cycle.status}")



[ADMIN] Step 1: Creating review cycle...


NotFoundError: Resource not found: http://127.0.0.1:8000/api/cycles

In [ ]:
# Step 2: Create form template
print("\n[ADMIN] Step 2: Creating form template...")
form = admin.forms.create(
    name="Performance Review Form",
    description="Standard performance review form",
)
print(f"   ✅ Created form: {form.id}")
print(f"   Name: {form.name}")

# Note: In a real scenario, you'd add fields to the form here
# For this demo, we assume the form already has fields or will be created via API


In [ ]:
# Step 3: Assign form to cycle
print("\n[ADMIN] Step 3: Assigning form to cycle...")
cycle = admin.cycles.update(cycle.id, form_template_id=form.id)
print(f"   ✅ Form assigned to cycle")
print(f"   Cycle form_template_id: {cycle.form_template_id}")


In [ ]:
# Step 4: Get employees for assignments
print("\n[ADMIN] Step 4: Getting employees...")
employees = admin.employees.list()
print(f"   Found {len(employees)} employees")

if len(employees) >= 3:
    reviewer_emp = employees[0]
    subject_emp = employees[1]
    approver_emp = employees[2]
    print(f"   Reviewer: {reviewer_emp.display_name} ({reviewer_emp.id})")
    print(f"   Subject: {subject_emp.display_name} ({subject_emp.id})")
    print(f"   Approver: {approver_emp.display_name} ({approver_emp.id})")
else:
    print("   ⚠️  Need at least 3 employees. Please create them first.")
    reviewer_emp = subject_emp = approver_emp = None


In [ ]:
# Step 5: Create review assignments
if reviewer_emp and subject_emp and approver_emp:
    print("\n[ADMIN] Step 5: Creating review assignment...")
    assignments = admin.assignments.bulk_create(
        cycle.id,
        [AssignmentCreate(
            reviewer_employee_id=reviewer_emp.id,
            subject_employee_id=subject_emp.id,
            approver_employee_id=approver_emp.id,
        )]
    )
    assignment = assignments[0]
    print(f"   ✅ Created assignment: {assignment.id}")
else:
    print("   ⚠️  Skipping assignment creation (need employees)")
    assignment = None


In [ ]:
# Step 6: Check cycle readiness
print("\n[ADMIN] Step 6: Checking cycle readiness...")
readiness = admin.cycles.check_readiness(cycle.id)
print(f"   Ready: {readiness.is_ready}")
print(f"   Can activate: {readiness.can_activate}")

if readiness.errors:
    print(f"   Errors: {readiness.errors}")
if readiness.warnings:
    print(f"   Warnings: {readiness.warnings}")
if readiness.checks:
    print(f"   Checks: {readiness.checks}")


In [ ]:
# Step 7: Activate cycle
if readiness.is_ready:
    print("\n[ADMIN] Step 7: Activating cycle...")
    cycle = admin.cycles.activate(cycle.id)
    print(f"   ✅ Cycle activated!")
    print(f"   Status: {cycle.status}")
else:
    print("\n   ⚠️  Cycle is not ready. Fix errors before activating.")


## 2. Reviewer Workflow: Complete Evaluation


In [ ]:
# Initialize reviewer client
reviewer = CycleManagerClient(
    base_url=BASE_URL,
    user_email=REVIEWER_EMAIL,
    api_prefix=API_PREFIX
)

print("👤 Reviewer client initialized")


In [ ]:
# Step 1: List my assignments
print("\n[REVIEWER] Step 1: Listing my assignments...")
assignments = reviewer.me.assignments(cycle_id=cycle.id)
print(f"   Found {len(assignments)} assignment(s)")

if assignments:
    assignment = assignments[0]
    print(f"   Using assignment: {assignment.id}")
else:
    print("   ⚠️  No assignments found. Cannot continue.")
    assignment = None


In [ ]:
# Step 2: Create evaluation
if assignment:
    print("\n[REVIEWER] Step 2: Creating evaluation...")
    evaluation = reviewer.evaluations.create(cycle.id, assignment.id)
    print(f"   ✅ Created evaluation: {evaluation.id}")
    print(f"   Status: {evaluation.status}")
    print(f"   Version: {evaluation.version}")
else:
    print("   ⚠️  Skipping evaluation creation (no assignment)")
    evaluation = None


In [ ]:
# Step 3: Save draft responses
if evaluation:
    print("\n[REVIEWER] Step 3: Saving draft responses...")
    # Example responses - adjust based on your form fields
    responses = {
        "rating": "5",
        "comments": "Excellent performance throughout the quarter.",
    }
    
    evaluation = reviewer.evaluations.save_draft(
        cycle.id,
        evaluation.id,
        responses,
    )
    print(f"   ✅ Draft saved")
    print(f"   Version: {evaluation.version}")
else:
    print("   ⚠️  Skipping draft save (no evaluation)")


In [ ]:
# Step 4: Validate evaluation
if evaluation:
    print("\n[REVIEWER] Step 4: Validating evaluation...")
    validation = reviewer.evaluations.validate(cycle.id, evaluation.id)
    print(f"   Valid: {validation.valid}")
    
    if validation.errors:
        print(f"   Errors: {[e.model_dump() for e in validation.errors]}")
    if validation.warnings:
        print(f"   Warnings: {validation.warnings}")
    
    if validation.valid:
        print("   ✅ Evaluation is valid and ready to submit!")
    else:
        print("   ⚠️  Evaluation has validation errors.")
else:
    print("   ⚠️  Skipping validation (no evaluation)")


In [ ]:
# Step 5: Submit evaluation
if evaluation and (not 'validation' in locals() or validation.valid):
    print("\n[REVIEWER] Step 5: Submitting evaluation...")
    evaluation = reviewer.evaluations.submit(cycle.id, evaluation.id)
    print(f"   ✅ Evaluation submitted!")
    print(f"   Status: {evaluation.status}")
    if evaluation.submitted_at:
        print(f"   Submitted at: {evaluation.submitted_at}")
else:
    print("   ⚠️  Skipping submission (evaluation not valid or doesn't exist)")


## 3. Approver Workflow: Review and Approve


In [ ]:
# Initialize approver client
approver = CycleManagerClient(
    base_url=BASE_URL,
    user_email=APPROVER_EMAIL,
    api_prefix=API_PREFIX
)

print("✅ Approver client initialized")


In [ ]:
# Step 1: List pending evaluations
print("\n[APPROVER] Step 1: Listing pending evaluations...")
evaluations = approver.me.evaluations(cycle_id=cycle.id, status="SUBMITTED")
print(f"   Found {len(evaluations)} pending evaluation(s)")

if evaluations:
    pending_eval = evaluations[0]
    print(f"   Reviewing evaluation: {pending_eval.id}")
else:
    print("   ⚠️  No pending evaluations found.")
    pending_eval = None


In [ ]:
# Step 2: Get evaluation details
if pending_eval:
    print("\n[APPROVER] Step 2: Getting evaluation details...")
    evaluation_detail = approver.evaluations.get(cycle.id, pending_eval.id)
    print(f"   ✅ Retrieved evaluation")
    print(f"   Status: {evaluation_detail.status}")
    print(f"   Responses: {evaluation_detail.responses}")
else:
    print("   ⚠️  Skipping evaluation details (no pending evaluation)")
    evaluation_detail = None


In [ ]:
# Step 3: Approve evaluation
if pending_eval:
    print("\n[APPROVER] Step 3: Approving evaluation...")
    approved_eval = approver.evaluations.approve(cycle.id, pending_eval.id)
    print(f"   ✅ Evaluation approved!")
    print(f"   Status: {approved_eval.status}")
    if approved_eval.approved_at:
        print(f"   Approved at: {approved_eval.approved_at}")
else:
    print("   ⚠️  Skipping approval (no pending evaluation)")


## 4. Statistics and Summary


In [ ]:
# Cycle statistics
print("\n" + "=" * 60)
print("CYCLE STATISTICS")
print("=" * 60)

stats = admin.cycles.get_stats(cycle.id)
print(f"\nCycle: {cycle.name}")
print(f"   Total assignments: {stats.total_assignments}")
print(f"   Total evaluations: {stats.total_evaluations}")
print(f"   Draft: {stats.draft_evaluations}")
print(f"   Submitted: {stats.submitted_evaluations}")
print(f"   Approved: {stats.approved_evaluations}")
print(f"   Returned: {stats.returned_evaluations}")
print(f"   Completion rate: {stats.completion_rate}%")


In [ ]:
# Reviewer statistics
print("\n" + "=" * 60)
print("REVIEWER STATISTICS")
print("=" * 60)

reviewer_stats = reviewer.me.stats()
print(f"\nTotal evaluations: {reviewer_stats.total_evaluations}")
print(f"   Draft: {reviewer_stats.draft_evaluations}")
print(f"   Submitted: {reviewer_stats.submitted_evaluations}")
print(f"   Approved: {reviewer_stats.approved_evaluations}")
print(f"   Returned: {reviewer_stats.returned_evaluations}")
print(f"\nTotal assignments: {reviewer_stats.total_assignments}")
print(f"   As reviewer: {reviewer_stats.reviewer_assignments}")
print(f"   As subject: {reviewer_stats.subject_assignments}")
print(f"   As approver: {reviewer_stats.approver_assignments}")


In [ ]:
# Approver statistics
print("\n" + "=" * 60)
print("APPROVER STATISTICS")
print("=" * 60)

approver_stats = approver.me.stats()
print(f"\nTotal evaluations: {approver_stats.total_evaluations}")
print(f"   Draft: {approver_stats.draft_evaluations}")
print(f"   Submitted: {approver_stats.submitted_evaluations}")
print(f"   Approved: {approver_stats.approved_evaluations}")
print(f"   Returned: {approver_stats.returned_evaluations}")
print(f"\nTotal assignments: {approver_stats.total_assignments}")
print(f"   As reviewer: {approver_stats.reviewer_assignments}")
print(f"   As subject: {approver_stats.subject_assignments}")
print(f"   As approver: {approver_stats.approver_assignments}")


## 5. Additional Examples


In [ ]:
# List all cycles
print("\nAll cycles:")
all_cycles = admin.cycles.list()
for c in all_cycles:
    print(f"   {c.name} ({c.status}) - {c.id}")


In [ ]:
# List all employees
print("\nAll employees:")
all_employees = admin.employees.list()
for emp in all_employees[:10]:  # Show first 10
    print(f"   {emp.display_name} ({emp.employee_number}) - {emp.id}")


In [ ]:
# Bulk employee lookup
if len(all_employees) >= 2:
    print("\nBulk employee lookup:")
    employee_ids = [all_employees[0].id, all_employees[1].id]
    result = admin.employees.bulk_lookup(employee_ids)
    print(f"   Found: {len(result.employees)} employees")
    print(f"   Missing: {len(result.missing_ids)} IDs")
    for emp in result.employees:
        print(f"     {emp.display_name}")


In [ ]:
# List form templates
print("\nForm templates:")
forms = admin.forms.list()
for form in forms:
    print(f"   {form.name} (v{form.version}) - Active: {form.is_active} - {form.id}")


In [ ]:
# Clean up - close clients
admin.close()
reviewer.close()
approver.close()

print("\n✅ All clients closed. Demo complete!")
